In [ ]:
import torch
import numpy as np

# 1) Load your q0 and k0 from the provided files
q_full = torch.load("subset_qk/block_1_q_proj_batch_6.pt")
k_full = torch.load("subset_qk/block_1_k_proj_batch_6.pt")
q = q_full[0]
k = k_full[0]
L, d_model = q.shape
num_heads = 32
d_head = d_model // num_heads

# Extract head 15
q0 = q.view(L, num_heads, d_head).permute(1, 0, 2)[15]
k0 = k.view(L, num_heads, d_head).permute(1, 0, 2)[15]
# Normalize as per the paper
q0 = q0 / (128**0.25)
k0 = k0 / (128**0.25)

# Convert to NumPy
q0_np = q0.cpu().numpy()
k0_np = k0.cpu().numpy()
V_np = (q0_np + 3* k0_np) /4

In [ ]:
nx, ny = 0, 0
for i in range(4096):
    nx_i = np.linalg.norm(q0_np[i,:])
    ny_i = np.linalg.norm(k0_np[i,:])
    if nx_i > nx:
        nx = nx_i
    if ny_i > ny:
        ny = ny_i


# Method2: Maclurain Random Feature implemented on one row of softmax

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

# Settings
P = 8
D = 2000
d = 128


sample = 4096

# Precompute phi_x
norm_x = np.zeros((sample))
phi_x = np.zeros((P, D))
w = np.where(np.random.rand(P, D, d) < 0.5, -1.0, +1.0)
pos = 200
x = np.array(q0[pos,:])
nx = 1
x/=nx
for p in range(1, P+1):
    for j in range(D):
        scale = 1 / np.sqrt(D)               
        for i in range(p):
            scale *= np.dot(w[i, j], x)
        phi_x[p-1, j] = scale 

            
approx_vals = [] 
for i in range(sample):
    y = np.array(k0[i,:]) 
    ny = 1
    y/=ny
    # Compute phi_y
    phi_y = np.zeros((P, D))
    for p in range(1, P+1):
        for j in range(D):
            scale = 1 / np.sqrt(D)
            for i in range(p):
                scale *= np.dot(w[i, j], y)
            phi_y[p-1, j] = scale
    # Approximate kernel
    approx_k = 1.0
    last_postive = 0.0
    for p in range(P):
        approx_k += abs(np.dot(phi_x[p], phi_y[p])) / math.factorial(p+1)
    approx_k = approx_k ** (nx * ny)
    approx_vals.append(approx_k)



In [ ]:

import numpy as np
import math
import matplotlib.pyplot as plt

# Settings
P = 8
D = 2000
d = 128


sample = 4096

# Precompute phi_x
phi_x_record= np.zeros((sample, P, D))
phi_x = np.zeros((P, D))
w = np.where(np.random.rand(P, D, d) < 0.5, -1.0, +1.0)
pos = 200

Z = (q0 + k0) / 2 # testing v
Z_100th = Z ** 0.01
# Let function f be row norm such that making row sum be 1 so f(exp) = softmax
# My implmention is softmax approxition without V(Z), softmax((Q) X dot (K)Y ) begin with exp(X dot Y) approxiamation using
# random feature where  exp(XdotY) = exp(X/10 dot Y/10) ** 100  =  exp(x, y) ** 100 and having a f on each row to get softmax approximation
# original function is softmax(X dot Y) Z = f(exp(X dot Y)) Z= f((phi x dot phi y) ** 100) dot Z   = f'(((phi x dot phi y dot Z_100th) ** 100))
# f' is not calculate the same way as f Please get it
# phi x dot phi y dot Z_100th can be calculate quickly as y dot Z_100th 's result can be reused
# make a power after all finished 

for i in range(sample):
    x = np.array(q0[pos,:])
    # 10 is scalar factor to make random feature valid
    x/=10
    for p in range(1, P+1):
        for j in range(D):
            scale = 1 / np.sqrt(D)               
            for ps in range(p):
                scale *= np.dot(w[ps, j], x)
            phi_x[p-1, j] = scale 
    phi_x_record[i] = phi_x

approx_record = np.zeros((sample, sample))            

for i in range(sample):
    y = np.array(k0[i,:]) 
    y/=10
    # Compute phi_y
    phi_y = np.zeros((P, D))
    for p in range(1, P+1):
        for j in range(D):
            scale = 1 / np.sqrt(D)
            for ps in range(p):
                scale *= np.dot(w[ps, j], y)
            phi_y[p-1, j] = scale
    
for i in range(sample):
    approx_vals = np.zeros((sample))
    for j in range(sample):
        # Approximate kernel
        approx_k = 1.0
        last_postive = 0.0
        for p in range(P):
            approx_k += abs(np.dot(phi_x[p], phi_y[p])) / math.factorial(p+1)
        approx_k = approx_k ** (100)
        approx_k /= approx_k.max()
        approx_k /= approx_k.sum()
        approx_vals[j] = approx_k
    approx_record[i] = approx_vals

# Normal calculation is approx_record product 


In [ ]:
x = np.array(q0[pos,:])
true_vals = []
sample = 40
for i in range(sample):
    y = np.array(k0[i,:]) 
    true_k = np.exp(np.dot(x, y))
    true_vals.append(true_k)

#### Normalized to ensure numerical stability

In [ ]:
true_vals /= max(true_vals)
true_vals /= sum(true_vals)

In [ ]:
approx_vals /= max(approx_vals)
approx_vals /= sum(approx_vals)

In [ ]:
approx_vals.shape

In [ ]:
torch.norm(torch.tensor(true_vals) - torch.tensor(approx_vals)) / torch.norm(torch.tensor(true_vals))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(40)
s1 = 0
begin, end = s1 * 40, s1*4000 + 400
sample = 4096
# 4) plot
plt.figure()
plt.plot(x[begin:end], true_vals[begin:end],    label='True exp(xᵀy)')
plt.plot(x[begin:end], approx_vals[begin:end],  label='Approx')
plt.ylabel('Kernel value')
plt.legend()
plt.show()

# Precompute all


In [156]:
import numpy as np
import math
import torch

# ——— Load and prepare q0, k0 as NumPy arrays ———
q_full = torch.load("subset_qk/block_1_q_proj_batch_6.pt", map_location="cpu")
k_full = torch.load("subset_qk/block_1_k_proj_batch_6.pt", map_location="cpu")

q = q_full[0]  # shape [L, d_model]
k = k_full[0]

L, d_model = q.shape
num_heads  = 32
d_head     = d_model // num_heads

# pick head 15 and first `sample` positions
sample = 4096
def sampling(q, k, sample):
    q0 = (
        q
        .view(L, num_heads, d_head)
        .permute(1, 0, 2)[15, :sample]
        .numpy()
    )   # shape [sample, d_head]
    k0 = (
        k
        .view(L, num_heads, d_head)
        .permute(1, 0, 2)[15, :sample]
        .numpy()
    )

    q0 = q0 / 128 ** 0.25
    k0 = k0 / 128 ** 0.25
    
    return q0, k0


def rfa_attention_fast(q0, k0, P=8, D=2000, shrink=10.0, power=100):
    """
    q0, k0:  (N, d) arrays of queries & keys
    returns: (N, N) approximate softmax(QK^T)
    """
    # 1) Pre-scale and cast to float32
    X = (q0 / shrink).astype(np.float64)   # (N, d)
    Y = (k0 / shrink).astype(np.float64)

    N, d = X.shape

    # 2) Sample ±1 weights in one flat block, cast to float32
    #    shape = (P*D, d)
    w_flat = np.sign(np.random.randn(P * D, d)).astype(np.float64)

    # 3) One mat-mul to get all projections, then reshape to (N,P,D)
    #    proj_x[n, p*D + j] = w_flat[p*D + j] ⋅ X[n]
    proj_x_flat = X.dot(w_flat.T)           # (N, P*D)
    proj_y_flat = Y.dot(w_flat.T)

    proj_x = proj_x_flat.reshape(N, P, D)   # (N, P, D)
    proj_y = proj_y_flat.reshape(N, P, D)

    # 4) Build per-degree normalizers √(D·p!) for p=1..P
    facts = np.array([math.sqrt(math.factorial(p+1)) for p in range(P)],
                     dtype=np.float64)     # (P,)
    normalizer = np.sqrt(D, dtype=np.float64) * facts
    normalizer = normalizer.reshape(1, P, 1)  # (1, P, 1)

    # 5) Cumulative product along the P-axis to get φ_p
    #    φ_p = ∏_{m=1..p} (proj[...,m-1] / normalizer[...,m-1])
    phi_x = np.cumprod(proj_x / normalizer, axis=1)  # (N, P, D)
    phi_y = np.cumprod(proj_y / normalizer, axis=1)

    # 6) Flatten φ back to (N, P*D)
    phi_x_flat = phi_x.reshape(N, P * D)
    phi_y_flat = phi_y.reshape(N, P * D)

    # 7) One big BLAS mat-mul to form the kernel matrix
    S = phi_x_flat.dot(phi_y_flat.T)  # (N, N)

    # 8) Sharpen & row-normalize
    M = (1.0 + S) ** power
    M /= M.max(axis=1, keepdims=True)
    M /= M.sum(axis=1, keepdims=True)

    return M


# usage:
# approx = rfa_attention_vectorized(q0, k0)

def true_softmax(q0, k0):
    dot = q0 @ k0.T
    true_val = np.exp(dot - dot.max(axis=1, keepdims=True))
    true_val /= true_val.sum(axis=1, keepdims=True)
    return true_val

def report_error(record_approx_values, true_val):
    return torch.norm(torch.tensor(record_approx_values - true_val)) / torch.norm(torch.tensor(true_val))



# Usage 

In [169]:
P, D, d = 4, 2000, 128
sample=4096
q0, k0 = sampling(q, k, sample)
v0 = q0

In [170]:
record_approx_values = rfa_attention_fast(q0, k0, P, D)

In [171]:
approx_v=record_approx_values @ v0

In [172]:
true_vals = true_softmax(q0, k0)

In [173]:
true_val = true_vals @ v0

In [174]:
import torch
torch.norm(torch.tensor(record_approx_values - true_vals)) / torch.norm(torch.tensor(true_vals))

tensor(0.2544, dtype=torch.float64)

In [175]:
import torch
torch.norm(torch.tensor(approx_v - true_val)) / torch.norm(torch.tensor(true_val))

tensor(0.0320, dtype=torch.float64)